# 5章: 係り受け解析
夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をCaboChaを使って係り受け解析し，その結果をneko.txt.cabochaというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

In [6]:
!cabocha -f1 < data/neko.txt >work/neko.txt.cabocha

## 目次
* [40. 係り受け解析結果の読み込み（形態素）](#sec1)
* [41. 係り受け解析結果の読み込み（文節・係り受け）](#sec2)
* [42. 係り元と係り先の文節の表示](#sec3)
* [43. 名詞を含む文節が動詞を含む文節に係るものを抽出](#sec4)
* [44. 係り受け木の可視化](#sec5)
* [45. 動詞の格パターンの抽出](#sec6)
* [46. 動詞の格フレーム情報の抽出](#sec7)
* [47. 機能動詞構文のマイニング](#sec8)
* [48. 名詞から根へのパスの抽出](#sec9)
* [49. 名詞間の係り受けパスの抽出](#sec10)  



In [1]:
!head -n30 ../work/neko.txt.cabocha

* 0 -1D 0/0 0.000000
一	名詞,数,*,*,*,*,一,イチ,イチ
EOS
EOS
* 0 2D 0/0 -0.764522
　	記号,空白,*,*,*,*,　,　,　
* 1 2D 0/1 -0.764522
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 2 -1D 0/2 0.000000
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。
EOS
* 0 2D 0/1 -1.911675
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 1 2D 0/0 -1.911675
まだ	副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ
* 2 -1D 0/0 0.000000
無い	形容詞,自立,*,*,形容詞・アウオ段,基本形,無い,ナイ,ナイ
。	記号,句点,*,*,*,*,。,。,。
EOS
EOS
* 0 1D 1/2 1.058678
　	記号,空白,*,*,*,*,　,　,　
どこ	名詞,代名詞,一般,*,*,*,どこ,ドコ,ドコ
で	助詞,格助詞,一般,*,*,*,で,デ,デ
* 1 4D 0/2 -1.453749


# 40. 係り受け解析結果の読み込み（形態素）<a id="sec1"></a>
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

インスタンス変数は後からバンバン定義できる  
slotsでインスタンス変数名を  
enum Cのenum  
classを単にメソッドのまとめ上げとして使うのは勿体無い(わかりにくい)  
Classを継承する際はhas a関係を意識  
Enum 値に名前をつけられる  
str.replaceは数が分かっている場合は指定した方が早い  
__init__のデフォルト引数にmutableな変数を渡すのはやめた方が良い  

[[]]*3 要素の複製なので、mutableを共有している(どれか変更したら全て変更される)  

parse関数を切り出す  
Morphのクラスメソッドとしてリストを作る関数を定義  


In [2]:
%%file q40.py
from itertools import groupby
from cytoolz import nth
file_path = 'work/neko.txt.cabocha'
class Morph:
    def __init__(self, surface, base, pos, pos1): 
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
        
    def __str__(self):
        return "{}\t{}, {}, {}".format(self.surface, self.base, self.pos, self.pos1)

    @classmethod
    def sent2morph(cls, sent_block): #文をmorphオブジェクトのリストに
        sentence = []
        for line in sent_block:
            if not line.startswith("* "):
                details = line.rstrip().replace("\t", ",").split(",")
                sentence.append(Morph(details[0], details[7], details[1], details[2]))
        return sentence
    
def parse():
    with open(file_path, 'r') as f:
        for is_eos, sent_block in groupby(f, lambda x: x.strip() == "EOS"): #文単位に分ける
            if not is_eos:
                yield Morph.sent2morph(sent_block)    
                
def main():
    neko = parse()
    for morph in nth(3, neko):
        print(morph)
            
if __name__ == "__main__":
    main()

Writing q40.py


In [3]:
!python q40.py

　	　, 記号, 空白
どこ	どこ, 名詞, 代名詞
で	で, 助詞, 格助詞
生れ	生れる, 動詞, 自立
た	た, 助動詞, *
か	か, 助詞, 副助詞／並立助詞／終助詞
とんと	とんと, 副詞, 一般
見当	見当, 名詞, サ変接続
が	が, 助詞, 格助詞
つか	つく, 動詞, 自立
ぬ	ぬ, 助動詞, *
。	。, 記号, 句点


# 41. 係り受け解析結果の読み込み（文節・係り受け）<a id="sec2"></a>
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

In [11]:
%%file q41.py
from q40 import Morph
from itertools import groupby
from cytoolz import nth
file_path = 'work/neko.txt.cabocha'

class Chunk(Morph):
    def __init__(self): 
        self.morphs = []
        self.dst = -1
        self.srcs = []
        self.index = 0

    def details(self): #結果出力用
        return ("<chunk{}> 係り先:{}, 係り元:" 
                + " ".join(str(index) for index in self.srcs) + "\n" 
                + "\n".join(str(morph) for morph in self.morphs) + "\n").format(self.index, self.dst)

    def __str__(self): #文節の文字列を返す
        return ("".join(morph.surface for morph in self.morphs if morph.pos != "記号")).strip()
    
    def surface_q49(self, alphabet): #名詞をalphabetに置き換えて表示(q49で利用)
        chunk_surface = ""
        for index_morph in range(len(self.morphs)):
            if self.morphs[index_morph].pos != "記号":
                if self.morphs[index_morph].pos == "名詞":
                    chunk_surface = chunk_surface + alphabet
                else:
                    chunk_surface = chunk_surface + self.morphs[index_morph].surface
        return chunk_surface
                                                            
    @classmethod
    def _dependency(cls, sentence): #係り元の文節のindexを探す
        for chunk in sentence:
            if chunk.dst > -1:
                sentence[chunk.dst].srcs.append(chunk.index)
            
    @classmethod
    def sent2chunk(cls, sent_block): #文をchunkのリストに
        sentence = []
        for is_chunk, chunk_or_morphs in groupby(sent_block, lambda x: x.startswith("*")):
            if is_chunk:
                details = list(chunk_or_morphs)[0].lstrip("*").strip().split()
                chunk = Chunk()
                chunk.dst = int(details[1].rstrip("D"))
                chunk.index = int(details[0])
            if not is_chunk:
                chunk.morphs = Morph.sent2morph(chunk_or_morphs) #親のメソッドで形態素のリストに
                sentence.append(chunk)
        cls._dependency(sentence)
        return sentence

def parse():
    with open(file_path, 'r') as f:
        for is_eos, sent_block in groupby(f, lambda x: x.strip() == "EOS"): #文単位に分ける
            if not is_eos:
                yield Chunk.sent2chunk(sent_block)
def main():
    neko = parse()
    for chunk in nth(7, neko):
        print(chunk.details())
        
if __name__ == "__main__":
    main()

Overwriting q41.py


In [12]:
!python q41.py

<chunk0> 係り先:1, 係り元:
この	この, 連体詞, *

<chunk1> 係り先:7, 係り元:0
書生	書生, 名詞, 一般
という	という, 助詞, 格助詞
の	の, 名詞, 非自立
は	は, 助詞, 係助詞

<chunk2> 係り先:4, 係り元:
時々	時々, 副詞, 一般

<chunk3> 係り先:4, 係り元:
我々	我々, 名詞, 代名詞
を	を, 助詞, 格助詞

<chunk4> 係り先:5, 係り元:2 3
捕え	捕える, 動詞, 自立
て	て, 助詞, 接続助詞

<chunk5> 係り先:6, 係り元:4
煮	煮る, 動詞, 自立
て	て, 助詞, 接続助詞

<chunk6> 係り先:7, 係り元:5
食う	食う, 動詞, 自立
という	という, 助詞, 格助詞

<chunk7> 係り先:-1, 係り元:1 6
話	話, 名詞, サ変接続
で	だ, 助動詞, *
ある	ある, 助動詞, *
。	。, 記号, 句点



# 42. 係り元と係り先の文節の表示<a id="sec3"></a>
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [28]:
%%file q42.py
from itertools import islice
from cytoolz import nth
from q41 import parse

def depend():
    for sentence in parse():
        sent_depend = []
        for chunk in sentence:
            if str(chunk) and chunk.dst != -1:
                sent_depend.append(str(chunk) + "\t" + str(sentence[chunk.dst]))
        yield sent_depend 

def main():
    for sent_depend in islice(depend(),7,8):
        for dependency in sent_depend:
            print(dependency)
            
if __name__ == "__main__":
    main()

Overwriting q42.py


In [29]:
!python q42.py

この	書生というのは
書生というのは	話である
時々	捕えて
我々を	捕えて
捕えて	煮て
煮て	食うという
食うという	話である


# 43. 名詞を含む文節が動詞を含む文節に係るものを抽出<a id="sec4"></a>
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [32]:
%%file q43.py
from itertools import islice
from q41 import parse

def is_pos(chunk, pos):
    for morph in chunk.morphs:
        if morph.pos == pos:
            return True
        
def depend():
    for sentence in parse():
        sent_depend = []
        for chunk in sentence:
            if all((str(chunk), chunk.dst != -1,
                    is_pos(chunk, "名詞"), is_pos(sentence[chunk.dst], "動詞"))):
                sent_depend.append(str(chunk) + "\t" + str(sentence[chunk.dst]))
        yield sent_depend 

def main():
    for sent_depend in islice(depend(),0,8):
        for dependency in sent_depend:
            print(dependency)

if __name__ ==  "__main__":
    main()

Overwriting q43.py


In [33]:
!python q43.py

どこで	生れたか
見当が	つかぬ
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している
吾輩は	見た
ここで	始めて
ものを	見た
あとで	聞くと
我々を	捕えて


# 44. 係り受け木の可視化<a id="sec5"></a>
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

In [36]:
%%file q44.py
from graphviz import Digraph
from q41 import parse
from cytoolz import nth

def make_tree(sentence):
    G = Digraph(format='png')
    G.attr('node', shape='circle')
    for chunk in sentence:
        surf_chunk = str(chunk)
        if surf_chunk: 
            G.node(str(chunk.index), surf_chunk)
            if chunk.dst != -1: 
                G.edge(str(chunk.index), str(chunk.dst))
    G.render('work/44')
    print(G)

def main():
    make_tree(nth(3, parse()))
    
if __name__ == "__main__":
    main()

Writing q44.py


In [37]:
!python q44.py

digraph {
	node [shape=circle]
	0 [label="どこで"]
		0 -> 1
	1 [label="生れたか"]
		1 -> 4
	2 [label="とんと"]
		2 -> 4
	3 [label="見当が"]
		3 -> 4
	4 [label="つかぬ"]
}


In [9]:
#<img src="../work/44.png"> で表示

<img src="work/44.png">

# 45. 動詞の格パターンの抽出<a id="sec6"></a>
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

* 動詞を含む文節において，最左の動詞の基本形を述語とする
* 述語に係る助詞を格とする
* 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる  

「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

始める  で  
見る    は を  
このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

* コーパス中で頻出する述語と格パターンの組み合わせ
* 「する」「見る」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

In [41]:
%%file q45.py
from itertools import chain
from q41 import parse

class VerbFrame:
    @classmethod
    def lget_verb(cls, chunk):
        for morph in chunk.morphs:
            if morph.pos == "動詞":
                return morph.base
        return False 
    
    @classmethod
    def get_zyosi(cls, chunk):
        list_zyosi = []
        for morph in chunk.morphs:
            if morph.pos == "助詞":
                list_zyosi.append(morph.base)
        return list_zyosi
    
    @classmethod
    def make_data(cls, chunk, sentence):
        return ([cls.lget_verb(chunk),[cls.get_zyosi(sentence[index_srcs]) 
                                       for index_srcs in chunk.srcs]])
    
    @classmethod
    def make_frame(cls):
        for sentence in parse():
            for chunk in sentence:
                if cls.lget_verb(chunk):
                    yield cls.make_data(chunk, sentence)
def main():
    with open("work/45.txt", "w") as f:
        for predicate in VerbFrame.make_frame():
            cases = " ".join(list(chain.from_iterable(predicate[1])))
            if cases:
                f.write(predicate[0] + "\t" + cases + "\n")

if __name__ == "__main__":
    main()

Overwriting q45.py


In [42]:
!python q45.py

In [43]:
!head -n10 work/45.txt

生れる	で
つく	か が
泣く	で
する	て だけ は
始める	で
見る	は を
聞く	で
捕える	を
煮る	て
食う	て


In [44]:
!cat work/45.txt | sort | uniq -c | sort -r 2>/dev/null | head

 565 云う	と
 442 する	を
 249 思う	と
 199 ある	が
 189 なる	に
 174 する	に
 173 見る	て
 127 する	と
 117 する	が
  94 見る	を


In [45]:
!cat work/45.txt | sort | uniq -c | sort -r | grep "する" 2>/dev/null | head

 442 する	を
 174 する	に
 127 する	と
 117 する	が
  84 する	て を
  59 する	は
  58 する	を に
  58 する	て
  51 する	が を
  48 する	から


In [46]:
!cat work/45.txt | sort | uniq -c | sort -r | grep "見る" 2>/dev/null | head

 173 見る	て
  94 見る	を
  21 見る	て て
  20 見る	から
  16 見る	て を
  14 見る	と
  12 見る	で
  11 見る	から て
  11 見る	は て
   8 見る	に


In [47]:
!cat work/45.txt | sort | uniq -c | sort -r | grep "与える" 2>/dev/null | head

   3 与える	に を
   1 与える	けれども に は を
   1 与える	じゃあ か と は て を
   1 与える	として を か
   1 与える	たり て に を
   1 与える	で だけ に を
   1 与える	に は に対して のみ は も
   1 与える	て が は は と て に を
   1 与える	は て に を に
   1 与える	は て に を


# 46. 動詞の格フレーム情報の抽出<a id="sec7"></a>
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

* 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
* 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる  
  
「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである.  
  
始める  で      ここで  
見る    は を   吾輩は ものを  

In [51]:
%%file q46.py
from itertools import chain
from q45 import VerbFrame

class VerbFrame2(VerbFrame):    
    @classmethod
    def get_chunk(cls, chunk):
        list_chunk = []
        for morph in chunk.morphs:
            if morph.pos == "助詞": 
                list_chunk.append(str(chunk))
        return list_chunk
    
    @classmethod
    def make_data(cls, chunk, sentence):
        return ([cls.lget_verb(chunk), 
                 [cls.get_zyosi(sentence[index_srcs]) for index_srcs in chunk.srcs],
                    [cls.get_chunk(sentence[index_srcs]) for index_srcs in chunk.srcs]])

def main():
    with open("work/46.txt", "w") as f:
        for predicate in VerbFrame2.make_frame():
            cases = " ".join(list(chain.from_iterable(predicate[1])))
            chunks = " ".join(list(chain.from_iterable(predicate[2])))
            if cases: 
                f.write(predicate[0] + "\t" + cases + " " + chunks + "\n")

if __name__ == "__main__":
    main()

Overwriting q46.py


In [52]:
!python q46.py

In [53]:
!head -n10 work/46.txt

生れる	で どこで
つく	か が 生れたか 見当が
泣く	で 所で
する	て だけ は 泣いて いた事だけは いた事だけは
始める	で ここで
見る	は を 吾輩は ものを
聞く	で あとで
捕える	を 我々を
煮る	て 捕えて
食う	て 煮て


# 47. 機能動詞構文のマイニング<a id="sec8"></a>
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．   

* 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
* 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
* 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
* 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）  
例えば「別段くるにも及ばんさと、主人は手紙に返事をする。」という文から，以下の出力が得られるはずである．
  
返事をする      と に は        及ばんさと 手紙に 主人は  
このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．  
  
* コーパス中で頻出する述語（サ変接続名詞+を+動詞）
* コーパス中で頻出する述語と助詞パターン

In [54]:
%%file q47.py
from itertools import chain
from q45 import VerbFrame

class VerbFrame3(VerbFrame): 
    @classmethod
    def get_zyosi(cls, chunk, sentence):
        list_zyosi = []
        list_chunk = []
        for index_chunk in chunk.srcs:
            morphs = sentence[index_chunk].morphs
            for morph_i in range(len(morphs) - 1):
                if morphs[morph_i].pos1 == "サ変接続" and  morphs[morph_i + 1].base == "を":
                    list_zyosi.append("を")
                    list_chunk.append(morphs[morph_i].surface + "を")
        if list_zyosi == []:
            return ""
        else:
            list_zyosi.append(list_chunk)
            return list_zyosi
    
    @classmethod
    def make_data(cls, chunk, sentence):
        return ([cls.lget_verb(chunk), cls.get_zyosi(chunk, sentence)])

def main():
    with open("work/47.txt", "w") as f:
        for predicate in VerbFrame3.make_frame():
            cases = " ".join(list(chain.from_iterable(predicate[1])))
            if cases:
                f.write(predicate[0] + "\t" + cases + "\n")

if __name__ == "__main__":
    main()

Writing q47.py


In [55]:
!python q47.py

In [56]:
!head -n20 work/47.txt

する	を 決心を
する	を 返報を
する	を 昼寝を
する	を 昼寝を
加える	を 迫害を
する	を 生活を
する	を 話を
する	を 投書を
する	を 話を
する	を 写生を
する	を 昼寝を
見る	を 彩色を
する	を 欠伸を
する	を 報道を
する	を 挨拶を
食う	を 御馳走を
する	を 問答を
する	を 雑談を
する	を 自慢を
飲み込む	を 呼吸を


In [57]:
!cut -f1 work/47.txt | sort | uniq -c | sort -r 2>/dev/null | head

 374 する
  21 やる
  18 聞く
  14 受ける
  11 願う
  10 始める
   9 加える
   9 致す
   8 与える
   7 告げる


In [58]:
!cat work/47.txt | sort | uniq -c |  sort -r 2>/dev/null | head

  30 する	を 返事を
  21 する	を 挨拶を
  16 する	を 話を
  14 する	を 真似を
  13 する	を 喧嘩を
   8 する	を 質問を
   7 する	を 運動を
   6 する	を 注意を
   6 する	を 昼寝を
   6 聞く	を 話を


# 48. 名詞から根へのパスの抽出<a id="sec9"></a>
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする.  
  
* 各文節は（表層形の）形態素列で表現する  
* パスの開始文節から終了文節に至るまで，各文節の表現を"->"で連結する  
「吾輩はここで始めて人間というものを見た」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．  

吾輩は -> 見た  
ここで -> 始めて -> 人間という -> ものを -> 見た  
人間という -> ものを -> 見た  
ものを -> 見た  

In [59]:
%%file q48.py
from q41 import parse
from itertools import islice
from cytoolz import nth

def exist_noun(chunk):
    for morph in chunk.morphs:
        if morph.pos == "名詞":
            return True
        
def chunk_is_root(chunk):
    if chunk.dst == -1:
        return True

def noun_to_root(sentence):
    roots = []
    for chunk in sentence:
        to_root = []
        if exist_noun(chunk):
            while(not(chunk_is_root(chunk))):
                to_root.append(chunk)
                chunk = sentence[chunk.dst]
            to_root.append(chunk)
            roots.append(to_root)
            to_root=[]
    return roots

def make_pathes_list(line_index):
    neko = parse()
    pathes = [[chunk for chunk in chunk_list if len(chunk_list) > 1] for chunk_list in noun_to_root(nth(line_index, neko))]
    return pathes

def main():
    for path in make_pathes_list(5):
        print("->".join([str(chunk) for chunk in path]))
    
if __name__ == "__main__":
    main()

Writing q48.py


In [60]:
!python q48.py

吾輩は->見た
ここで->始めて->人間という->ものを->見た
人間という->ものを->見た
ものを->見た


# 49. 名詞間の係り受けパスの抽出<a id="sec10"></a>
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がi
とj
（i<j
）のとき，係り受けパスは以下の仕様を満たすものとする．

* 問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を"->"で連結して表現する
* 文節i
とj
に含まれる名詞句はそれぞれ，XとYに置換する  
  
  
また，係り受けパスの形状は，以下の2通りが考えられる．

* 文節i
から構文木の根に至る経路上に文節j
が存在する場合: 文節i
から文節j
のパスを表示
* 上記以外で，文節i
と文節j
から構文木の根に至る経路上で共通の文節k
で交わる場合: 文節i
から文節k
に至る直前のパスと文節j
から文節k
に至る直前までのパス，文節k
の内容を"|"で連結して表示  
  
例えば，「吾輩はここで始めて人間というものを見た。」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

Xは | Yで -> 始めて -> 人間という -> ものを | 見た  
Xは | Yという -> ものを | 見た  
Xは | Yを | 見た  
Xで -> 始めて -> Y  
Xで -> 始めて -> 人間という -> Y  
Xという -> Y  



In [63]:
%%file q49.py
from q48 import make_pathes_list
from itertools import combinations

def rule(path1, path2):
    for index_chunk1 in range(len(path1)):
        for index_chunk2 in range(len(path2)):
            if path1[index_chunk1] == path2[index_chunk2]:
                k = path1[index_chunk1] #文節k
                i_to_k = [str(path1[i_ch1]) if i_ch1 > 0 else path1[i_ch1].surface_q49("X")
                          for i_ch1 in range(index_chunk1)]
                j_to_k = [str(path2[i_ch2]) if i_ch2 > 0 else path2[i_ch2].surface_q49("Y")
                          for i_ch2 in range(index_chunk2)]
                return [i_to_k, j_to_k, k]
def main():
    pathes = make_pathes_list(5)
    pathes_q49 = [] #抽出されたパスのリスト
    for path_comb in combinations(pathes, 2):
        pathes_q49.append(rule(*path_comb))
    for path_q49 in pathes_q49:
        if isinstance(path_q49, type(None)): pass
        elif not(len(path_q49[0]) and len(path_q49[1])) :
            print(" -> ".join(path_q49[0]) + " -> ".join(path_q49[1]) + " -> " + path_q49[2].surface_q49("Y"))
        else:
            print("->".join(path_q49[0]) + " | " + " -> ".join(path_q49[1]) + " | " + str(path_q49[2]))

if __name__ =="__main__":
    main()

Overwriting q49.py


In [64]:
!python q49.py

Xは | Yで -> 始めて -> 人間という -> ものを | 見た
Xは | Yという -> ものを | 見た
Xは | Yを | 見た
Xで -> 始めて -> Yという
Xで -> 始めて -> 人間という -> Yを
Xという -> Yを
